# INTELIGENCIA ARTIFICIAL - EVALUACIÓN 1
Integrantes: Sebastian Racedo y Sebastian Valle

In [ ]:
# Libraries Import
import math
import pandas as pd

In [ ]:
# Read and split data into training and test sets
def SplitData(Raw_Data):
    data = pd.read_csv(Raw_Data,sep = ';')
    [n,m] = data.shape
    X = data.iloc[:,0:m-1]
    y = data.iloc[:,m-1]
    #print(X.shape)
    return X,y

In [ ]:
[X,y] = SplitData("data.csv")
X_data = pd.DataFrame(X.values.reshape((X.shape[0],X.shape[1])))
y_data = pd.DataFrame(y.values.reshape((y.shape[0], 1))) 

In [ ]:
y

In [ ]:
y_data

In [ ]:
def Entropy(ydata):
    Size_data = len(ydata)
    if Size_data == 0:
        return 0
    num_classes = [0 for i in ydata.iloc[:,0].dropna().value_counts()]   
    num_classes = ydata.iloc[:,0].dropna().value_counts().tolist()
    num_classes = [x/Size_data for x in num_classes]
    ent = 0
    for num in num_classes:
        ent += num*math.log(num,2)
    return num*-1

In [ ]:
a = Entropy(ydata)
print(a)

In [ ]:
# All_Data = X , Subsets = disjoint subsets of All_Data
def GainRatio(All_Data, Subsets):
    Size_data = len(All_Data)
    # Original Entropy
    Original_Entropy = Entropy(All_Data)
    Weights_Per_Subsets = [len(x)/Size_data for x in Subsets]
    Entropy_After_Branch = 0
    for x in range(len(Subsets)):
        Entropy_After_Branch += Weights_Per_Subsets[x]*Entropy(Subsets[x])
    Total_Gain  = Original_Entropy - Entropy_After_Branch
    return Total_Gain